In [1]:
!pip install -U folium

In [2]:
import pandas as pd
import folium
import branca
%matplotlib inline

# Because autocomplete was taking too long ...
%config Completer.use_jedi = False

In [3]:
df = (
    pd.read_csv("../data.csv")
    .rename(columns={
        "Property.Address.Longitude": "long",
        "Property.Address.Latitude": "lat",
        "Price/SizeInternal_frozen": "price_per_size",
    })
)

In [4]:
df.head()

,long,lat,Price,SizeInternal,price_per_size,Building.BathroomTotal,Building.Bedrooms,Building.StoriesTotal,Building.Type,Property.Price,...,Property.TypeId,Property.ZoningType,Property.AmmenitiesNearBy,Property.ParkingType,PostalCode,Building.SizeExterior,Property.Parking[1].Name,Property.OwnershipType,Property.Parking[2].Name,Building.UnitTotal
0,-73.578687,45.489532,689000,738.00,933,2,3,20.0,Apartment,"$689,000",...,300,Residential,"Highway, CEGEP, Hospital, Park, Public Transit...",NaN,H3H0B3,NaN,NaN,Condominium/Strata,NaN,NaN
1,-73.574184,45.464970,635000,856.80,741,2,5,2.0,Apartment,"$635,000",...,300,Residential,"Highway, Hospital, Park, Schools, Public Transit",NaN,H4G1G8,NaN,NaN,Condominium/Strata,NaN,NaN
2,-73.558671,45.494718,699000,956.91,730,2,3,197.0,Apartment,"$699,000",...,300,Residential,"Highway, Hospital, Park, Public Transit","Garage, Integrated Garage",H3C0P3,NaN,Integrated Garage,Condominium/Strata,NaN,NaN
3,-73.737033,45.535935,660000,935.35,705,2,5,1.0,House,"$660,000",...,300,Residential,"Highway, CEGEP, Hospital, Park, Schools",Other,H1V1H8,7243.47 sqft,NaN,NaN,NaN,NaN
4,-73.559654,45.542168,669000,990.00,675,2,3,7.0,Apartment,"$669,000",...,300,Residential,"Highway, CEGEP, Hospital, Park, Schools, Publi...",Garage,H1W0B9,NaN,NaN,Condominium/Strata,NaN,NaN


In [5]:
# Check if we have missing geo coodinates
print(len(df[df["long"].isna() | df["lat"].isna()]))

# Check if we have missing price
print(len(df[df["Price"].isna() | df["price_per_size"].isna()]))

0
0


In [6]:
df.iloc[0]

long                                                                     -73.5787
lat                                                                       45.4895
Price                                                                      689000
SizeInternal                                                                  738
price_per_size                                                                933
Building.BathroomTotal                                                          2
Building.Bedrooms                                                               3
Building.StoriesTotal                                                          20
Building.Type                                                           Apartment
Property.Price                                                           $689,000
Property.Type                                                       Single Family
Property.Address.AddressText    2020 Boul. René-Lévesque O.|#911|Montréal (Vil...
Property.Parking

In [7]:
base_map = folium.Map(location=[df.iloc[0]["lat"], df.iloc[0]["long"]], zoom_start = 11)

In [8]:
colorscale = branca.colormap.linear.YlOrRd_09.scale(df["Price"].min(), df["Price"].max())
colorscale.caption = "Color based on absolute prices"
colorscale.add_to(base_map)

In [9]:
for index, row in df.iterrows():
    folium.CircleMarker(
        [row["lat"], row["long"]],
        popup=row["Price"],
        fill=True,
        fill_opacity=0.7,
        fill_color=colorscale(row["Price"]),
        color=colorscale(row["Price"]),
    ).add_to(base_map)
base_map

In [10]:
df_with_price_per_size = df[df["price_per_size"] != "#ERR"].astype({"price_per_size": int})

In [11]:
base_map = folium.Map(location=[df_with_price_per_size.iloc[0]["lat"], df_with_price_per_size.iloc[0]["long"]], zoom_start = 11)

colorscale = branca.colormap.linear.YlOrRd_09.scale(df_with_price_per_size["price_per_size"].min(), df_with_price_per_size["price_per_size"].max())
colorscale.caption = "Color based on prices per size"
colorscale.add_to(base_map)

In [12]:
for index, row in df_with_price_per_size.iterrows():
    folium.CircleMarker(
        [row["lat"], row["long"]],
        popup=row["price_per_size"],
        fill=True,
        fill_opacity=0.7,
        fill_color=colorscale(row["price_per_size"]),
        color=colorscale(row["price_per_size"]),
    ).add_to(base_map)
base_map